**File to test performance with MLP s.t. we can compare LSTM performance**

for inputs, we can just take our input dataframe from the LSTM script, and remove the temporal data (columns t1-t20).
we then must include the historical volatility. so inputs are the same as black scholes. current price, strike price, 
implied volatility, risk free rate, time to expiry.

**Get Data**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

inputs_df = pd.read_csv('data/input_data.csv')

display(inputs_df.head())

call_df = inputs_df[inputs_df['call_put'] == "Call"]
call_df = call_df.drop(['call_put'], axis=1)
put_df = inputs_df[inputs_df['call_put'] == "Put"]
put_df = put_df.drop(['call_put'], axis=1)

CALL_X_train, CALL_X_test, CALL_Y_train, CALL_Y_test = train_test_split(call_df.loc[:,['currentP', 'strike', 'vol', 'days_expiry', 'r']], call_df.loc[:,['option_fp']], 
                                                                        test_size=.1, random_state=1)

PUT_X_train, PUT_X_test, PUT_Y_train, PUT_Y_test = train_test_split(put_df.loc[:,['currentP', 'strike', 'vol', 'days_expiry', 'r']], put_df.loc[:,['option_fp']], 
                                                                        test_size=.1, random_state=1)

print(CALL_X_train.shape)

,currentP,strike,call_put,vol,days_expiry,r,t1,t2,t3,t4,...,t12,t13,t14,t15,t16,t17,t18,t19,t20,option_fp
0,170.41,145.0,Call,0.4236,13,2.42,153.8,152.29,150.0,153.07,...,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,25.675
1,170.41,145.0,Call,0.4236,13,2.42,153.8,152.29,150.0,153.07,...,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,25.675
2,170.41,145.0,Call,0.4236,13,2.42,153.8,152.29,150.0,153.07,...,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,25.675
3,170.41,145.0,Call,0.4236,13,2.42,153.8,152.29,150.0,153.07,...,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,25.675
4,170.41,145.0,Call,0.4236,13,2.42,153.8,152.29,150.0,153.07,...,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,25.675


(59490, 5)


**Model Architecture**

In [3]:
features = 5
layers = 4
n_units = 400
n_batch = 4096

In [4]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model

model = Sequential()
model.add(Dense(n_units, input_dim=CALL_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer=Adam())

2024-09-06 09:22:34.100625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 400)            │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 400)            │       160,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 400)            │         1,600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 400)            │       160,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 400)            │         1,600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 400)            │       160,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 400)            │         1,600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 488,801 (1.86 MB)

 Trainable params: 486,401 (1.86 MB)

 Non-trainable params: 2,400 (9.38 KB)

In [6]:
history = model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - loss: 61.7558 - val_loss: 170.3651
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 3.4988 - val_loss: 273.2636
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 2.1157 - val_loss: 737.8782
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - loss: 1.6228 - val_loss: 626.2686
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - loss: 1.3461 - val_loss: 390.6686
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - loss: 1.1732 - val_loss: 181.8860
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - loss: 1.1041 - val_loss: 55.7523
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 1.0158 - val_loss: 29.0291
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - loss: 0.9003 - val_loss: 18.9398
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - loss: 0.8145 - val_loss: 26.8870


In [23]:
model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
history = model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=4096, epochs=20, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 101ms/step - loss: 0.6931 - val_loss: 42.4062
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.5930 - val_loss: 43.6676
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.5627 - val_loss: 52.4136
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 0.5566 - val_loss: 26.2173
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.5580 - val_loss: 8.8229
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.4884 - val_loss: 11.4830
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.4836 - val_loss: 4.1867
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - loss: 0.4626 - val_loss: 3.0147
Epoch 9/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.4283 - val_loss: 2.8360
Epoch 10/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - loss: 0.4449 - val_loss: 1.3983
Epoch 11/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.4283 - val_loss: 2.0053
Epoch 12/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - los

In [24]:
model.compile(loss='mse', optimizer=Adam(learning_rate=1e-4))
history = model.fit(CALL_X_train, CALL_Y_train, 
                    batch_size=4096, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - loss: 0.4915 - val_loss: 2.0116
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 0.4233 - val_loss: 1.5044
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 0.4016 - val_loss: 0.5545
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - loss: 0.4103 - val_loss: 0.6662
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 0.4326 - val_loss: 0.7757
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.4011 - val_loss: 0.5318
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 0.3929 - val_loss: 0.6908
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.3675 - val_loss: 0.7821
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 0.3756 - val_loss: 0.4782
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.3637 - val_loss: 1.1838
